In [2]:
%pip install openmeteo-requests
%pip install numpy pandas


[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
import openmeteo_requests as openmeteo
import numpy as np
import pandas as pd

In [32]:
latitude = 13.7563
longitude = 100.5018

start_date = "2021-08-01"
end_date = "2025-01-31"

In [36]:
openmeteo = openmeteo_requests.Client()

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": start_date,
    "end_date": end_date,
    "daily": ["rain_sum", "precipitation_hours", "precipitation_sum"],
    "timezone": "Asia/Bangkok",
}
responses = openmeteo.weather_api(url, params=params)

In [37]:
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 13.743409156799316°N 100.49586486816406°E
Elevation 4.0 m asl
Timezone b'Asia/Bangkok'b'GMT+7'
Timezone difference to GMT+0 25200 s


In [39]:
daily = response.Daily()
daily_rain_sum = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()

daily_data = {
    "date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
    )
}

daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_sum"] = daily_precipitation_sum

rain_df = pd.DataFrame(data=daily_data)

                          date  rain_sum  precipitation_hours  \
0    2021-07-31 17:00:00+00:00       3.2                  6.0   
1    2021-08-01 17:00:00+00:00       1.1                  6.0   
2    2021-08-02 17:00:00+00:00       5.9                  4.0   
3    2021-08-03 17:00:00+00:00       2.9                  7.0   
4    2021-08-04 17:00:00+00:00       4.9                  8.0   
...                        ...       ...                  ...   
1275 2025-01-26 17:00:00+00:00       0.2                  1.0   
1276 2025-01-27 17:00:00+00:00       0.0                  0.0   
1277 2025-01-28 17:00:00+00:00       0.0                  0.0   
1278 2025-01-29 17:00:00+00:00       0.0                  0.0   
1279 2025-01-30 17:00:00+00:00       0.0                  0.0   

      precipitation_sum  
0                   3.2  
1                   1.1  
2                   5.9  
3                   2.9  
4                   4.9  
...                 ...  
1275                0.2  
1276       

In [42]:
rain_df.head(5)

,date,rain_sum,precipitation_hours,precipitation_sum
0,2021-07-31 17:00:00+00:00,3.2,6.0,3.2
1,2021-08-01 17:00:00+00:00,1.1,6.0,1.1
2,2021-08-02 17:00:00+00:00,5.9,4.0,5.9
3,2021-08-03 17:00:00+00:00,2.9,7.0,2.9
4,2021-08-04 17:00:00+00:00,4.9,8.0,4.9


In [40]:
rain_df.describe()

,rain_sum,precipitation_hours,precipitation_sum
count,1280.000000,1280.000000,1280.000000
mean,4.964766,5.274219,4.964766
std,7.602226,5.592687,7.602226
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.300000,4.000000,1.300000
75%,7.100000,8.000000,7.100000
max,56.800003,24.000000,56.800003


In [41]:
rain_df.to_csv("rain_data.csv", index=False)